In [1]:
!pip install requests

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from a .env file
NEWS_API_KEY = os.getenv("3585d350c03446288e395f48c373fbf2")

api_key = "3585d350c03446288e395f48c373fbf2"

In [3]:
import requests

def get_top_news_topics(city_name, api_key, num_topics=5):
    url = "https://newsapi.org/v2/everything"
    params = {
        'q': city_name,   # Query for news related to the city name
        'apiKey': api_key,
        'pageSize': num_topics,
        'language': 'en',  # Adjust as needed for your target audience
        'sortBy': 'relevancy'  # Sort results by relevance
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Check for request errors
        data = response.json()
        
        # Extract top topics if available
        topics = [article['title'] for article in data.get('articles', [])]
        return topics if topics else ["No news topics found."]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news data: {e}")
        return []


In [4]:
# Usage
city = "Pokhara"
api_key = "3585d350c03446288e395f48c373fbf2"  # or directly paste your key here for testing

top_news_topics = get_top_news_topics(city, api_key)
print("Top trending news topics:", top_news_topics)


Top trending news topics: ['The dangers of altitude sickness and how to avoid it', 'AK Monthly Recap: September 2024', '‘Cats of the World’ Is the Perfect Book for Feline-Loving Travelers', 'Budget-friendly international trips from India', 'Nepal protesters clash with police over politician’s fraud charges']


**Explanation of Key Parts**

Parameters in params:

* q: The city name for the search query.
* pageSize: Limits the number of articles returned (default is 5).
* language: Restricts results to English (en) or another language if desired.
* sortBy: Options include relevancy, popularity, and publishedAt.
* Error Handling:

The try-except block ensures the function catches any errors with the request (e.g., network issues, invalid API keys).
Returning Topics:

* The function extracts the title of each article, which contains the headline, and returns a list of the top 5 headlines.
* This code will fetch and print the top 5 trending news topics for the specified city, which will serve as input for your next steps in gathering discussions from public forums.

In [5]:
!pip install praw


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.3 MB/s eta 0:00:00a 0:00:01


In [8]:
import praw
import json

# Set up Reddit API client
def initialize_reddit():
    reddit = praw.Reddit(
        client_id="YOUR_CLIENT_ID",
        client_secret="YOUR_CLIENT_SECRET",
        user_agent="city_news_discussion_collector"
    )
    return reddit

# Fetch Reddit discussions for each topic
def get_reddit_discussions(reddit, topics, num_posts=5, num_comments=3):
    discussions = {}
    for topic in topics:
        posts = []
        for submission in reddit.subreddit("all").search(topic, limit=num_posts):
            comments = []
            submission.comments.replace_more(limit=0)  # Expand comments tree
            
            # Fetch the top comments
            for comment in submission.comments[:num_comments]:
                comments.append(comment.body)
            
            post = {
                "title": submission.title,
                "text": submission.selftext,
                "comments": comments
            }
            posts.append(post)
        
        discussions[topic] = posts
    return discussions


In [9]:

# Example usage
reddit = initialize_reddit()
topics = ["Topic 1", "Topic 2", "Topic 3", "Topic 4", "Topic 5"]  # Replace with topics from Step 1
reddit_discussions = get_reddit_discussions(reddit, topics)

# Print or store the discussions in JSON format
print(json.dumps(reddit_discussions, indent=2))


ResponseException: received 401 HTTP response